In [20]:
import numpy as np
import pandas as pd
from tensorflow import keras
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,Dropout,BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from keras.utils import np_utils
import os
import random
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Add, Dense, Activation, BatchNormalization, Conv2D
from keras.utils import np_utils
from keras.initializers import glorot_uniform
%matplotlib inline


In [21]:
!mkdir ../data
!wget -O ../data/fer2013.csv https://www.dropbox.com/s/zi48lkarsg4kbry/fer2013.csv\?dl\=1

mkdir: cannot create directory ‘../data’: File exists
--2022-11-22 11:42:08--  https://www.dropbox.com/s/zi48lkarsg4kbry/fer2013.csv?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/zi48lkarsg4kbry/fer2013.csv [following]
--2022-11-22 11:42:08--  https://www.dropbox.com/s/dl/zi48lkarsg4kbry/fer2013.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc90ad563e8ef315c07c4945d55a.dl.dropboxusercontent.com/cd/0/get/BxOwsj0vNHJMx-Fyq7Rcwv9gCe84ICPCNIh5J37r0iZIDGTpGW2lzg4aivt9DyvSl68CgSUJPRK8eRdzqbfTC5iR50IWT4IAhKZTk6uWa02B6bh5mpkMvu14xW2U3R_Hm72TsXoTki9TsxTWUUV0srhs9DdmQMwhOqjnIl60RQKiMw/file?dl=1# [following]
--2022-11-22 11:42:09--  https://uc90ad563e8ef315c07c4945d55a.dl.dropboxusercontent.com/cd/0/get/BxOwsj0vNHJMx-Fyq7Rcwv9gCe8

In [22]:
num_classes=2
input_shape=(48,48,1)
model_name = "fer2013"
model_type = "fer2013"
epochs = 200
lrate = 0.01
decay = lrate/epochs
batch_size = 32


In [23]:

def load_dataset(net=True):
    """Utility function to load the FER2013 dataset.
    
    It returns the formated tuples (X_train, y_train) , (X_test, y_test).

    Parameters
    ==========
    net : boolean
        This parameter is used to reshape the data from images in 
        (cols, rows, channels) format. In case that it is False, a standard
        format (cols, rows) is used.
    """

    # Load and filter in Training/not Training data:
    df = pd.read_csv('../data/fer2013.csv')
    training = df.loc[df['Usage'] == 'Training']
    testing = df.loc[df['Usage'] != 'Training']

    # X_train values:
    X_train = training[['pixels']].values
    X_train = [np.fromstring(e[0], dtype=int, sep=' ') for e in X_train]
    if net:
        X_train = [e.reshape((48, 48, 1)).astype('float32') for e in X_train]
    else:
        X_train = [e.reshape((48, 48)) for e in X_train]
    X_train = np.array(X_train)

    # X_test values:
    X_test = testing[['pixels']].values
    X_test = [np.fromstring(e[0], dtype=int, sep=' ') for e in X_test]
    if net:
        X_test = [e.reshape((48, 48, 1)).astype('float32') for e in X_test]
    else:
        X_test = [e.reshape((48, 48)) for e in X_test]
    X_test = np.array(X_test)

    # y_train values:
    y_train = training[['emotion']].values
    # y_train = keras.utils.to_categorical(y_train)

    # y_test values
    y_test = testing[['emotion']].values
    # y_test = keras.utils.to_categorical(y_test)

    return (X_train, y_train) , (X_test, y_test)

In [ ]:
(X_train, y_train) , (X_test, y_test) = load_dataset()
print(y_train.shape, y_test.shape)
print(X_train.shape, X_test.shape)

In [ ]:
class_names=["Angry","Disgust","Fear","Happy","Sad","Surprise","Neutral"]
plt.style.use('ggplot')

plt.figure(figsize=(12, 12))
for i in range(16):
    j = random.randint(0,28709) 
    plt.subplot(4, 4, i + 1)
    plt.imshow(X_train[j].reshape((48, 48)), cmap="gray")
    plt.title("{}".format(class_names[y_train[j][0]]), fontsize=12)
    plt.axis('off')
    plt.tight_layout()


In [ ]:
class_3_index = np.where(y_train.reshape(-1) == 3)
X_train_class_3 = X_train[class_3_index]
y_train_class_3 = y_train[class_3_index]



class_4_index = np.where(y_train.reshape(-1) == 4)
X_train_class_4 = X_train[class_4_index]
y_train_class_4 = y_train[class_4_index]



X_train = np.concatenate((X_train_class_3, X_train_class_4))
y_train = np.concatenate((y_train_class_3, y_train_class_4)).reshape(-1,1)

# ---------------------------------------------------------------------------

class_3_index = np.where(y_test.reshape(-1) == 3)
X_test_class_3 = X_test[class_3_index]
y_test_class_3 = y_test[class_3_index]

class_4_index = np.where(y_test.reshape(-1) == 4)
X_test_class_4 = X_test[class_4_index]
y_test_class_4 = y_test[class_4_index]


X_test = np.concatenate((X_test_class_3, X_test_class_4))
y_test = np.concatenate((y_test_class_3, y_test_class_4)).reshape(-1,1)


print(y_train.shape, y_test.shape)
print(X_train.shape, X_test.shape)


In [ ]:
print("__________________train_____________________")
plt.figure(figsize=(12, 12))
for i in range(16):
    j = random.randint(0,12045) 
    plt.subplot(4, 4, i + 1)
    plt.imshow(X_train[j].reshape((48, 48)), cmap="gray")
    plt.title("{}".format(class_names[y_train[j][0]]), fontsize=12)
    plt.axis('off')
    plt.tight_layout()


In [ ]:
print("___________________________________test___________________________")
plt.figure(figsize=(12, 12))
for i in range(16):
    j = random.randint(0,3021) 
    plt.subplot(4, 4, i + 1)
    plt.imshow(X_test[j].reshape((48, 48)), cmap="gray")
    plt.title("{}".format(class_names[y_test[j][0]]), fontsize=12)
    plt.axis('off')
    plt.tight_layout()

In [ ]:
y_train = np.where(y_train > 3, 1, 0)
y_test = np.where(y_test > 3, 1, 0)
class_names = ['Happy',  'Sad']

print(y_train.shape, y_test.shape)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

X = np.concatenate((X_test, X_train))
Y = np.concatenate((y_test, y_train)).reshape(-1,1)

# X, y = shuffle(X, Y, random_state=0)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15)

print(y_train.shape, y_test.shape)
print(X_train.shape, X_test.shape)



In [ ]:

print("__________________train_____________________")
plt.figure(figsize=(12, 12))
for i in range(16):
    j = random.randint(0,12806) 
    plt.subplot(4, 4, i + 1)
    plt.imshow(X_train[j].reshape((48, 48)), cmap="gray")
    plt.title("{}".format(class_names[y_train[j][0]]), fontsize=12)
    plt.axis('off')
    plt.tight_layout()


In [ ]:
print("___________________________________test___________________________")
plt.figure(figsize=(12, 12))
for i in range(16):
    j = random.randint(0,2260) 
    plt.subplot(4, 4, i + 1)
    plt.imshow(X_test[j].reshape((48, 48)), cmap="gray")
    plt.title("{}".format(class_names[y_test[j][0]]), fontsize=12)
    plt.axis('off')
    plt.tight_layout()

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.
X_test = X_test / 255.

X_train_mean = np.mean(X_train)
X_train -= X_train_mean
X_test_mean = np.mean(X_test)
X_test -= X_test_mean

y_train = np_utils.to_categorical(y_train,)
y_test = np_utils.to_categorical(y_test,)
y_train = np.delete(y_train,1,1)
y_test = np.delete(y_test,1,1)

print(y_train.shape, y_test.shape)
print(X_train.shape, X_test.shape)

In [ ]:
# print("__________________train_____________________")
# plt.figure(figsize=(12, 12))
# for i in range(16):
#     j = random.randint(0,12045) 
#     plt.subplot(4, 4, i + 1)
#     plt.imshow(X_train[j].reshape((48, 48)), cmap="gray")
#     plt.title("{}".format(class_names[int(y_train[j])]), fontsize=12)
#     plt.axis('off')
#     plt.tight_layout()

In [ ]:
# print("___________________________________test___________________________")
# plt.figure(figsize=(12, 12))
# for i in range(16):
#     j = random.randint(0,3021) 
#     plt.subplot(4, 4, i + 1)
#     plt.imshow(X_test[j].reshape((48, 48)), cmap="gray")
#     plt.title("{}".format(class_names[y_test[j][0]]), fontsize=12)
#     plt.axis('off')
#     plt.tight_layout()

In [ ]:
import seaborn as sns

plt.title('Distribution')
sns.countplot(y = 0, data=pd.DataFrame(y_train))
plt.show()

In [ ]:
from tensorflow.keras.layers import Dropout
def vgg16(input_shape=(48,48,1), num_classes=2):
  model = Sequential()
  model.add(Conv2D(input_shape=input_shape,filters=64,kernel_size=(3,3),padding="same", activation="relu"))
  model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))


  model.add(Dropout(0.8))

  model.add(Conv2D(filters=8, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(Conv2D(filters=8, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

  model.add(Conv2D(filters=16, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(Conv2D(filters=16, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(Conv2D(filters=16, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

  # model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
  # model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
  # model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
  # model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

  # model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
  # model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
  # model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
  # model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

  model.add(Flatten())
  model.add(Dense(units=4096,activation="relu"))
  model.add(Dense(units=4096,activation="relu"))
  model.add(Dense(units=1, activation="sigmoid"))

  return model

In [ ]:
def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


In [ ]:
# Prepare callbacks for model saving and for learning rate adjustment.

early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, verbose=0, mode='auto', baseline=None, restore_best_weights=True)



lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [ lr_reducer, lr_scheduler,early_stopping]


In [ ]:
model = vgg16(input_shape,num_classes)

# Compile model

optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(optimizer=optimizer,
              loss=keras.losses.BinaryCrossentropy(label_smoothing = 0.1),
              metrics=[keras.metrics.BinaryAccuracy()])
print("Model Summary of ",model_type)
print(model.summary())



In [ ]:
# Score trained model.
history = model.fit(X_train, y_train,
      batch_size=batch_size,
      epochs=epochs,
      validation_data=(X_test, y_test),
      shuffle=True,
      callbacks=callbacks)
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['binary_accuracy'])
plt.plot(history.history['val_binary_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc

y_pred = model.predict(X_test).ravel()

nn_fpr_keras, nn_tpr_keras, nn_thresholds_keras = roc_curve(y_test, y_pred)
auc_keras = auc(nn_fpr_keras, nn_tpr_keras)
plt.plot(nn_fpr_keras, nn_tpr_keras, marker='.', label='Neural Network (auc = %0.3f)' % auc_keras)
plt.title('ROC Curve')
plt.ylabel('TPR')
plt.xlabel('FPR')
plt.show()
plt.axis('off')


print(auc_keras)

In [ ]:
from matplotlib import pyplot
predict_x=model.predict(X_test) 

happy_index = np.where(y_test.reshape(-1) == 0.)
happy = predict_x[happy_index]

sad_index = np.where(y_test.reshape(-1) == 1.)
sad = predict_x[sad_index]



bins = np.linspace(0, 1, 100)

pyplot.hist(happy, bins, alpha=0.5, label='happy')
pyplot.hist(sad, bins, alpha=0.5, label='sad')
pyplot.legend(loc='upper right')
pyplot.show()

In [ ]:
len(happy)


In [ ]:
len(sad)

In [ ]:
# for X,Y in zip(predict_x,y_test):
#     print("Model Score:", X, "actual score:", Y) 